In [3]:
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials

In [5]:
import pandas as pd
import requests
import chardet

In [6]:
cred = credentials.Certificate("cred.json")
#firebase_admin.initialize_app(cred)

In [7]:
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://dsci551-c4bee-default-rtdb.firebaseio.com/"
})


In [5]:
data = {"user":{
          "student1":{
              "name": "Vicky",
              "Age": 21
          },
          "student2":{
              "name": "Kun",
              "Age": 24
          },
          "student3":"haha"
        }
       }
ref = db.reference('/')
ref.set(data)

In [8]:
import json

In [9]:
def ls(dir_in):
    dir_list = []
    try:
        ref1 = db.reference(dir_in)
        result = ref1.get()
        if result == None:
            return'No directory exists'
        else:
            for i in result:
                dir_list.append(i)
    except:
        return'Invalid error'
    return dir_list

In [10]:
# check the directory before the last one
def dirExist(dir_in):
    try:
        if_exist = True
        if dir_in[0] != '/':
            return False
        dir_in_list = dir_in.split('/')
        dir_in_list = dir_in_list[1:]
        for i in range(len(dir_in_list)):
            if i != (len(dir_in_list) - 1):
                current_dir = '/' + '/'.join(dir_in_list[0:i])
                current_ls = ls(current_dir)
                if dir_in_list[i] not in current_ls:
                    if_exist = False
                    break
        return if_exist
    except:
        return False

In [11]:
def newDirExist(dir_in):
    try:
        if_exist = True
        if dir_in[0] != '/':
                return False
        dir_in_list = dir_in.split('/')
        dir_in_list = dir_in_list[1:]  
        last_ls = ls('/' + '/'.join(dir_in_list[0:-1]))
        if dir_in_list[-1] not in last_ls:
            if_exist = False
        return if_exist
    except:
        return False

In [12]:
def mkDirectory(dir_in):
    try:
        dir_in_list = dir_in.split('/')
        dir_in_list = dir_in_list[1:]
        if dirExist(dir_in) == False:
            return 'Directory not exist'
        if newDirExist(dir_in) == True:
            return 'You new diretory already exists'
        if len(dir_in_list) >= 2:
            child_ref = dir_in_list[-2]
            new_ref = dir_in_list[-1]
            dir_ref = dir_in[0: -(len(child_ref) + len(new_ref) + 2)]
            mkdir = db.reference(dir_ref)
            child = mkdir.child(child_ref)
            child.update({new_ref: ""})
        else:
            new_ref = dir_in_list[-1]
            dir_ref = '/'
            mkdir = db.reference(dir_ref)
            mkdir.update({new_ref: ""})
    except:
        return'Invalid error'
    return 'You successfully made a new directory'

In [30]:
def uploadFile(fileName):
    # if filename not found?
    try: 
        if fileName not in ['collision.csv', 'parties.csv', 'victims.csv', 'LA_Weather.csv', 'LA_County_COVID_Cases.csv']:
            return 'File not found'
        if dirExist('/Data/x') == False:
            d = mkDirectory('/Data')
        if fileName == 'collision.csv':
            df = pd.read_csv('../Data/' + fileName, encoding='latin1',low_memory = False)
            df['collision_date'] = df['collision_date'].astype('string')
            df1 = df[df['collision_date'].str.contains('2020') == True]
            json1 = df1.to_json(orient = 'index')
            df2 = df[df['collision_date'].str.contains('2021') == True]
            json2 = df2.to_json(orient = 'index')
            df1_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/collision_json/" + "collision_2020.json"
            df2_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/collision_json/" + "collision_2021.json"
            requests.put(df1_url, json1)
            requests.put(df2_url, json2)
            return [df1_url, df2_url]
        elif fileName == 'parties.csv':
            df = pd.read_csv('../Data/' + fileName, encoding='latin1')
            df1 = df[df['at_fault'] == 0]
            json1 = df1.to_json(orient = 'index')
            df2 = df[df['at_fault'] == 1]
            json2 = df2.to_json(orient = 'index')
            df1_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/parties_json/" + "parties_0.json"
            df2_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/parties_json/" + "parties_1.json"
            requests.put(df1_url, json1)
            requests.put(df2_url, json2)
            return [df1_url, df2_url]
        elif fileName == 'victims.csv':
            df = pd.read_csv('../Data/' + fileName, encoding='latin1')
            json = df.to_json(orient = 'index')
            df_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/victims_json/" + "victims.json"
            requests.put(df_url, json)
            return [df_url]
        elif fileName == 'LA_Weather.csv':
            df = pd.read_csv('../Data/' + fileName, encoding='latin1')
            df['datetime'] = df['datetime'].astype('str')
            df1 = df[df['datetime'].str.contains('2020') == True]
            json1 = df1.to_json(orient = 'index')
            df2 = df[df['datetime'].str.contains('2021') == True]
            json2 = df2.to_json(orient = 'index')
            df1_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/LA_Weather_json/" + "LA_Weather_2020.json"
            df2_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/LA_Weather_json/" + "LA_Weather_2021.json"
            requests.put(df1_url, json1)
            requests.put(df2_url, json2)
            return [df1_url, df2_url]
        elif fileName == 'LA_County_COVID_Cases.csv':
            df = pd.read_csv('../Data/' + fileName, encoding='latin1')
            df['date'] = df['date'].astype('str')
            df1 = df[df['date'].str.contains('2020') == True]
            json1 = df1.to_json(orient = 'index')
            df2 = df[df['date'].str.contains('2021') == True]
            json2 = df2.to_json(orient = 'index')
            df1_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/LA_County_COVID_Cases_json/" + "LA_Covid_2020.json"
            df2_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/LA_County_COVID_Cases_json/" + "LA_Covid_2021.json"
            requests.put(df1_url, json1)
            requests.put(df2_url, json2)
            return [df1_url, df2_url]
    except:
        return 'Invalid Input'

In [31]:
def put(new_file, dir_in):
    try:
        dir_in_list = dir_in.split('/')
        dir_in_list = dir_in_list[1:]
        new_file_json = new_file.replace('.csv', '_json')
        if dirExist(dir_in + '/' + new_file) == False:
            return 'Directory not exist'
        if newDirExist(dir_in + '/' + new_file_json) == True:
            return 'You file already exists'
        child_ref = dir_in_list[-1]
        dir_ref = dir_in[0: -(len(child_ref) + 1)]
        if dir_ref == '':
            dir_ref = '/'
        result_url = uploadFile(new_file)
        if result_url == 'Invalid Input' or result_url == 'File not found':
            return 'Invalid Input'
        mkdir = db.reference(dir_ref)
        child = mkdir.child(child_ref)
        child.update({new_file_json: ""})
        child = child.child(new_file_json)
        if len(result_url) == 2:
            child.update({"p1": result_url[0]})
            child.update({"p2": result_url[1]})
        else:
            child.update({"p": result_url[0]})
        return "You successfully created a file"
    except:
        return "Invalid Input"

In [15]:
def rm(dir_in):
    try:
        if dirExist(dir_in + '/x') == False:
                return 'Directory not exist'
        ref = db.reference(dir_in)
        ref.delete()
        return "Successfully deleted"
    except:
        return "Invalid Input"

In [16]:
def cat(dir_in):
    try:
        if dirExist(dir_in) == True and newDirExist(dir_in) == True:      
            ref = db.reference(dir_in)
            par = ref.get()
            dir_in = dir_in.split('/')
            if dir_in[-1] == 'LA_Weather_json':
                url1 = par['p1']
                response1 = requests.get(url1)
                r1 = response1.json()
                while None in r1:
                    r1.remove(None)
                d1 = json.dumps(r1)
                df1 = pd.read_json(d1)

                url2 = par['p2']
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df2 = pd.read_json(d2, orient = 'index')

                frames = [df1, df2]
                df = pd.concat(frames)
            if dir_in[-1] == 'victims_json':
                url1 = par['p']
                response1 = requests.get(url1)
                r1 = response1.json()
                while None in r1:
                    r1.remove(None)
                d1 = json.dumps(r1)
                df = pd.read_json(d1)
            if dir_in[-1] == 'parties_json':
                url1 = par['p1'] 
                response1 = requests.get(url1)
                r1 = response1.json()
                df1 = pd.DataFrame()
                for a in r1:
                    df1 = df1.append(a, ignore_index = True)

                url2 = par['p2'] 
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df2 = pd.read_json(d2, orient = 'index')

                frames = [df1, df2]
                df = pd.concat(frames)
            if dir_in[-1] == 'collision_json':      
                url1 = par['p1'] 
                response1 = requests.get(url1)
                r1 = response1.json()
                df1 = pd.DataFrame()
                for a in r1:
                    df1 = df1.append(a, ignore_index = True)
                    
                url2 = par['p2']
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df2 = pd.read_json(d2, orient = 'index')

                frames = [df1, df2]
                df = pd.concat(frames)
            if dir_in[-1] == 'LA_County_COVID_Cases_json':
                url1 = par['p1']
                response1 = requests.get(url1)
                r1 = response1.json()
                d1 = json.dumps(r1)
                df1 = pd.read_json(d1, orient = 'index')

                url2 = par['p2']
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df2 = pd.read_json(d2, orient = 'index')
                

                frames = [df1, df2]
                df = pd.concat(frames)
            return df
        else:
            return "File not exist"
    except:
        return "Invalid Input"

In [17]:
def getPartitionLocations(dir_in):
    try:
        if dirExist(dir_in) == True and newDirExist(dir_in) == True:
            dir_in_list = dir_in.split('/')
            if dir_in_list[-1] != 'victims_json':
                if dir_in_list[-1] == 'LA_Weather_json':
                    v1 = 'LA_Weather_2020'
                    v2 = 'LA_Weather_2021'
                elif dir_in_list[-1] == 'LA_Country_COVID_Cases_json':
                    v1 = 'LA_Covid_2020'
                    v2 = 'LA_Covid_2021'
                elif dir_in_list[-1] == 'collision_json':
                    v1 = 'collision_2020'
                    v2 = 'collision_2021'
                elif dir_in_list[-1] == 'parties_json':
                    v1 = 'parties_0'
                    v2 = 'parties_1'
                return ['/Data/' + dir_in_list[-1] + '/' + v1, '/Data/' + dir_in_list[-1] + '/' + v2]
            else:
                return '/Data/' + dir_in_list[-1] + '/p'
        else:
            return 'Directory not exist'
    except:
        return 'Invalid Input'

In [43]:
def readPartition(dir_in, num):
    try:
        if dirExist(dir_in) == True and newDirExist(dir_in) == True:
            ref = db.reference(dir_in)
            par = ref.get()
            dir_in = dir_in.split('/')
            if dir_in[-1] == 'LA_Weather_json' and num == 1:
                url1 = par['p1']
                response1 = requests.get(url1)
                r1 = response1.json()
                while None in r1:
                    r1.remove(None)
                d1 = json.dumps(r1)
                df = pd.read_json(d1)
                return df
            elif dir_in[-1] == 'LA_Weather_json' and num == 2:
                url2 = par['p2']
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df = pd.read_json(d2, orient = 'index')
                return df
            elif dir_in[-1] == 'victims_json' and num == 1:
                url1 = par['p']
                response1 = requests.get(url1)
                r1 = response1.json()
                while None in r1:
                    r1.remove(None)
                d1 = json.dumps(r1)
                df = pd.read_json(d1)
                return df
            elif dir_in[-1] == 'parties_json' and num == 1:
                url1 = par['p1'] 
                response1 = requests.get(url1)
                r1 = response1.json()
                df = pd.DataFrame()
                for a in r1:
                    new_df = pd.DataFrame([a])
                    df = pd.concat([df, new_df], axis=0, ignore_index=True)
                return df
            elif dir_in[-1] == 'parties_json' and num == 2:
                url2 = par['p2'] 
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df = pd.read_json(d2, orient = 'index')
                return df
            elif dir_in[-1] == 'collision_json' and num == 1:
                url1 = par['p1'] 
                response1 = requests.get(url1)
                r1 = response1.json()
                df = pd.DataFrame()
                for a in r1:
                    new_df = pd.DataFrame([a])
                    df = pd.concat([df, new_df], axis=0, ignore_index=True)
                return df
            elif dir_in[-1] == 'collision_json' and num == 2:
                url2 = par['p2']
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df = pd.read_json(d2, orient = 'index')
                return df
            elif dir_in[-1] == 'LA_County_COVID_Cases_json' and num == 1:
                url1 = par['p1']
                response1 = requests.get(url1)
                r1 = response1.json()
                d1 = json.dumps(r1)
                df = pd.read_json(d1, orient = 'index')
                return df
            elif dir_in[-1] == 'LA_County_COVID_Cases_json' and num == 2:
                url2 = par['p2']
                response2 = requests.get(url2)
                r2 = response2.json()
                d2 = json.dumps(r2)
                df = pd.read_json(d2, orient = 'index')
                return df
            else:
                return 'Invalid Input'
        else:
            return 'Directory not exist'
    except:
        return 'Invalid Input'

# Add

In [19]:
a = mkDirectory('/user')
a

'You successfully made a new directory'

In [34]:
b = put('victims.csv', '/root/user/dataset')
b

'You file already exists'

In [144]:
c = uploadFile('parties.csv')
c

['https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/parties_json/parties_0.json',
 'https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/parties_json/parties_1.json']

In [42]:
readPartition('/root/user/dataset/collision_json', 1)

,0,beat_number,beat_type,bicycle_collision,bicyclist_injured_count,bicyclist_killed_count,case_id,chp_beat_class,chp_beat_type,chp_road_type,...,postmile,side_of_highway,state_route,weather_2,postmile_prefix,ramp_intersection,road_condition_2,secondary_ramp,primary_ramp,pcf_violation_code
0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,1,chp county roadarea,0.0,0.0,0.0,91166585.0,chp other,county road area,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,132T1,not chp,0.0,0.0,0.0,9193735.0,not chp,not chp,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,1,not chp,0.0,0.0,0.0,9099765.0,not chp,not chp,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,2,not chp,0.0,0.0,0.0,81326866.0,not chp,not chp,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,None,116,chp state highway,0.0,0.0,0.0,91255414.0,chp other,state route,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19996,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997,None,82,chp state highway,0.0,0.0,0.0,91218818.0,chp other,interstate,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998,None,52,chp state highway,0.0,0.0,0.0,91275697.0,chp other,interstate,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
df = pd.read_csv('../Data/' + 'collision.csv', encoding='latin1',low_memory = False)
df['collision_date'] = df['collision_date'].astype('string')
df1 = df[df['collision_date'].str.contains('2020') == True]
json1 = df1.to_json(orient = 'index')
df
# df1_url = "https://dsci551-c4bee-default-rtdb.firebaseio.com/Data/collision_json/" + "collision_2020.json"
# requests.put(df1_url, json1)

,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,county_location,special_condition,beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
0,91482038,9535,22319,NaN,2200 thru 0559,unincorporated,1900,los angeles,0,chp county roadline,...,0,0,0,NaN,NaN,34.02486,-118.17390,2021/5/16,1:20:00,2021/5/27
1,91420426,9575,17491,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,34.15603,-118.29962,2021/2/24,16:00:00,2021/3/4
2,9216184,1900,606776,530,not chp,50000 to 100000,1959,los angeles,0,not chp,...,0,0,0,NaN,NaN,34.08772,-118.07346,2021/1/16,11:40:00,2021/2/22
3,91441191,9530,22581,NaN,2200 thru 0559,unincorporated,1900,los angeles,0,chp county roadarea,...,0,0,0,NaN,NaN,33.89616,-118.19228,2021/1/17,22:30:00,2021/4/1
4,9238022,1941,11550,NaN,not chp,>250000,1941,los angeles,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2021/3/5,14:55:00,2021/4/19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9259752,1900,611140,2608,not chp,100000 to 250000,1950,los angeles,0,not chp,...,0,0,0,NaN,NaN,34.55829,-118.05853,2021/4/27,18:20:00,2021/6/1
19996,9220927,1942,30851,2138,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2021/1/28,2:00:00,2021/3/11
19997,91467423,9535,22245,NaN,1400 thru 2159,50000 to 100000,1943,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,33.92560,-118.17769,2021/4/28,21:52:00,2021/5/7
19998,91420563,9590,21040,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,34.02345,-118.20088,2021/3/3,14:30:00,2021/3/4


In [263]:
df = pd.read_csv('../Data/' + 'collision.csv', encoding='latin1',low_memory = False)
df['collision_date'] = df['collision_date'].astype('string')
df1 = df[df['collision_date'].str.contains('2020') == True]
df1

,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,county_location,special_condition,beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
1,91166585,9530,21310,NaN,1400 thru 2159,unincorporated,1900,los angeles,0,chp county roadarea,...,0,0,0,NaN,NaN,33.98938,-118.24752,2020/1/9,17:00:00,2020/1/16
2,9193735,1900,647602,1323,not chp,50000 to 100000,1935,los angeles,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2020/11/11,10:30:00,2021/1/5
3,9099765,1924,80434,15,not chp,50000 to 100000,1924,los angeles,0,not chp,...,0,0,0,NaN,NaN,33.90192,-118.32398,2020/4/30,8:55:00,2020/7/21
4,81326866,1933,1017,15,not chp,100000 to 250000,1933,los angeles,0,not chp,...,0,0,0,NaN,NaN,33.95883,-118.36163,2020/10/10,18:13:00,2020/10/20
5,91254469,9535,22305,NaN,1400 thru 2159,50000 to 100000,1969,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,33.94545,-118.17057,2020/6/10,14:03:00,2020/6/19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9081473,1942,43464,507,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2020/3/11,17:10:00,2020/6/22
19995,91255414,9580,21954,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,34.27467,-118.53783,2020/6/20,19:25:00,2020/6/22
19997,91218818,9590,22405,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,34.09591,-118.24416,2020/3/23,18:45:00,2020/3/30
19998,91275697,9590,22431,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,34.05074,-118.26182,2020/7/19,18:50:00,2020/7/28
